In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

## 予測をする

In [ ]:
# 予測を行う。一つの関数・変数（コア数・各種ベンチマーク由来の初期化変数）

benchmarkNames = ["cg"]
classes = ["S", "W", "A", "B", "C", "D", "E", "F"]
processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]

# データを取得
rawDF = returnCollectedExistingData(
    benchmarkNames=benchmarkNames,
    classes=classes,
    processes=processes,
    csvDirPath="./csv_files/",
)
rawDF = addInitDataToRawDF(rawDF)

# データを表示
print(rawDF)

# TODO:予測を実施


# TODO:予測の統計的結果をグラフにプロットする